# Core import

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import seaborn as sns
import pandas as pd
import os
import pathlib
import shutil
import cv2
import sys

%cd adas_system/notebooks

try:
    USE_TPU = bool(os.environ['COLAB_TPU_ADDR'])
except:
    USE_TPU = False

if USE_TPU:
    # !pip uninstall pytorch
    # !pip install cloud-tpu-client==0.10 torch==1.10.0
    # !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
    !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
    import torch_xla
    import torch_xla.core.xla_model as xm
    USE_TPU = True

else:
    USE_TPU = False

IN_COLAB = False
import torch
from torch import nn

try:
    import google.colab
    IN_COLAB = True
    from google.colab import drive
    drive.mount('/content/drive')
    if not os.path.isfile('1_ClassifierResearch.ipynb'):
        print('already exist')
        !git clone --branch 9_SignDetector https://github.com/lsd-maddrive/adas_system.git
        %cd adas_system/notebooks
        !mkdir ../data/rtsd-frames
        !unzip -j -q /content/drive/MyDrive/USER_FULL_FRAMES.zip -d ./../data/rtsd-frames
        !pwd
        !ls

except:
    if IN_COLAB:
        !git clone --branch 9_SignDetector https://github.com/lsd-maddrive/adas_system.git
        !gdown --id 1npYrZB_1WsgPv3cCrhv8KbfZuTvTxWow
        %cd adas_system/notebooks
        !mkdir ../data/rtsd-frames
        !unzip -j -q /content/USER_FULL_FRAMES.zip -d ./../data/rtsd-frames
    else:
        pass

###
sys.path.append('D:\\d_tsw\\main_diplom\\notebooks\\SignDetectorAndClassifier\\src')
from helper_utils import *
###

TEXT_COLOR = 'black'

# Зафиксируем состояние случайных чисел
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
random.seed(RANDOM_STATE)
%matplotlib inline
plt.rcParams["figure.figsize"] = (17,10)

if USE_TPU:
    device = xm.xla_device()
else:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# Init dirs

In [ ]:
if not IN_COLAB:
    PROJECT_ROOT = pathlib.Path(os.path.join(os.curdir, os.pardir))
else:
    PROJECT_ROOT = pathlib.Path('..')

PROJECT_ROOT = pathlib.Path('D:\\d_tsw\\main_diplom\\SignDetectorAndClassifier')
DATA_DIR = PROJECT_ROOT / 'data'
NOTEBOOKS_DIR = PROJECT_ROOT / 'notebooks'
VIDEO_DIR = DATA_DIR / 'reg_videos'

video_path = str(VIDEO_DIR / '1.mp4')
print(video_path)

In [ ]:
PROJECT_ROOT

In [ ]:
import logging
from datetime import datetime
now = datetime.now

LARGE = True

if LARGE:
    restore = DATA_DIR / 'YoloV5L.pt'
    model_cfg_file = DATA_DIR / 'yolov5l_custom_anchors.yaml'
else:
    restore = DATA_DIR / 'YoloV5.pt'
    model_cfg_file = DATA_DIR / 'yolov5s_custom_anchors.yaml'

logger = logging.getLogger()
# logger.setLevel('WARN')
model = Model(cfg=model_cfg_file, ch=3, nc=1);
model.load_state_dict(torch.load(restore, map_location='cpu'))
# logger.setLevel('INFO')

torch.cuda.empty_cache()
detectInterface = makeDetectFromModel(model)

if not detectInterface.device == device:
    detectInterface.to(device)

In [ ]:
LARGE

In [ ]:
from IPython.display import display, Image
from utils.augmentations import letterbox

'''
frame - BRG input frame
trained_img_size - img.shape on which Model trained
'''
def FastTransformImgToModelInputFormat(frame, trained_img_size=(640, 640)):
    frame = letterbox(frame, img_size, auto=False)[0]
    frame = frame.transpose((2, 0, 1))[::-1]
    frame = np.ascontiguousarray(frame)
    frame = torch.from_numpy(frame).float()
    frame /= 255
    frame = frame[None, ...]
    return frame

video = cv2.VideoCapture(video_path)
display_handle1=display(1, display_id=True)
display_handle2=display(2, display_id=True)

if LARGE:
    img_size = 416
else:
    img_size = 640
    
img_size = (img_size, img_size)

torch.cuda.empty_cache()

i = 1
while True:
    i += 1
    if i % 2:
        for j in range(3):
            _, frame_src = video.read()
        continue
        
    t0 = now()
    
    
    _, frame_src = video.read()
    frame_marks = frame_src.copy()
    
    cv2.imshow('frame_src', frame_src)
    cv2.waitKey()
    assert False
    
    modelInput = FastTransformImgToModelInputFormat(frame_marks, trained_img_size=img_size)
    pred = detectInterface(modelInput.to(device))
    
    data = detectInterface.translatePreds(pred, img_size, frame_src.shape, conf_thres=0.101, max_det=10)
    
    for i in range(data['count']):
        COORD_ARR = [data['coords'][i][0], 
                     data['coords'][i][1], 
                     data['coords'][i][2], 
                     data['coords'][i][3]
                    ]
        
        frame_marks = cv2.rectangle(frame_marks, (COORD_ARR[0], COORD_ARR[1]), 
                        (COORD_ARR[2], COORD_ARR[3]), 
                        (255, 0, 0), 
                        3)
        
        frame_marks = cv2.putText(frame_marks, str(round(data['confs'][i], 3)), 
                           (data['coords'][i][2] - 40, data['coords'][i][3]),
                           cv2.FONT_HERSHEY_SIMPLEX,
                           0.7, (255, 255, 0),
                           5, cv2.LINE_AA
                          )
        
        frame_marks = cv2.putText(frame_marks, str(round(data['confs'][i], 3)), 
                           (data['coords'][i][2] - 40, data['coords'][i][3]),
                           cv2.FONT_HERSHEY_SIMPLEX,
                           0.7, (0, 0, 0),
                           2, cv2.LINE_AA
                          )
    dt = now() - t0
    frame_src = cv2.putText(frame_src, 
                            'fps:' + str(round(1 / dt.total_seconds() , 2)),
                            (0, 60),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            3, (0, 0, 0),
                            3, cv2.LINE_AA
                           )
    _, frame_src = cv2.imencode('.jpeg', frame_src)
    _, frame_marks = cv2.imencode('.jpeg', frame_marks)
   
    display_handle1.update(Image(data=frame_src.tobytes()))
    display_handle2.update(Image(data=frame_marks.tobytes()))
    
    torch.cuda.empty_cache()

In [ ]:
cv2.destroyAllWindows()